In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
cmtes = ScrapedCommittee.objects.filter(committee_group='federal', office__icontains='house')
races = {'congress': []}
for cmte in cmtes:
    if cmte.office not in races:
        races[cmte.office] = []
    found = False
    for cand in races[cmte.office]:
        if cand.filer_id == cmte.filer_id:
            found = True
    if not found:
        races[cmte.office].append(cmte)
        races['congress'].append(cmte)

In [3]:
from csv import DictWriter

for race in races:
    print race
    contribs = ScrapedContribution.objects.filter(filing_committee__filer_id__in=[c.filer_id for c in races[race]])
    values = contribs.values('donor', 'city', 'state', 'employer', 'occupation', 'filing_committee__name').annotate(
        sum=Sum('amount'), num=Count('date')).order_by('-amount', '-sum')
    refunds = ScrapedRefund.objects.filter(
        filing_committee__filer_id__in=[c.filer_id for c in races[race]]).values(
        'donor', 'city', 'state').annotate(sum=Sum('amount'))
    refund_dict = {}
    for refund in refunds:
        key = '-'.join([refund['donor'], refund['city'], refund['state']])
        refund_dict[key] = refund
        refund_dict[key]['uses'] = 0
    output = []
    for donor in values[:2000]:
        key = '-'.join([
                donor['donor'] if donor['donor'] else '',
                donor['city'] if donor['city'] else '',
                donor['state'] if donor['state'] else ''])
        if key in refund_dict:
            if refund_dict[key]['uses'] < 1:
                donor['sum'] -= refund_dict[key]['sum']
                refund_dict[key]['uses'] += 1
        output.append(donor)
    with open('/home/aepton/code/campfin/data/%s_donors.csv' % race.replace(' ', '').lower(), 'w+') as fh:
        writer = DictWriter(fh, sorted(output[0].keys()))
        writer.writeheader()
        writer.writerows(sorted(output, key=lambda k: k['sum'], reverse=True))

congress


IOError: [Errno 2] No such file or directory: '/home/aepton/code/campfin/data/congress_donors.csv'